In [107]:
import warnings

import pandas as pd

from jre_utils.datapath import (
    factor_data_paths,
    get_derived_csv_path,
    get_derived_lpa_path,
    get_derived_plps_path,
)
from jre_utils.config import asset_types
from jre_utils.visualize import plot_time_series
from jre_utils.process import (
    get_most_active_municipalities,
    get_highest_growth_municipalities,
    get_cumulative_growth,
)

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [108]:
asset_type = "building"

factor = "taxable_income"
factor_column = "taxable_income_growth"

metrics = {
    "weighted_mean": "unit_price_wmean",
    "weighted_median": "unit_price_wmedian",
    "mean": "unit_price_mean",
    "median": "unit_price_median",
}

dataset_paths = {
    "main": get_derived_csv_path(asset_type),
    "lpa": get_derived_lpa_path(),
    "plps": get_derived_plps_path()
}

In [109]:
granularity_columns = ["area", "area_code"]
group_by_columns = granularity_columns + ["year"]
display_columns = ["unit_price", "total_traded_area", "count"]

metric = metrics["weighted_median"]
metric_pct_chg = metric + "_pct_chg"
upcoming_metric = "upcoming_" + metric
upcoming_metric_pct_chg = "upcoming_" + metric_pct_chg

In [110]:
core_df_path = dataset_paths["main"]
factor_path = factor_data_paths["processed"][factor]["municipality"]

df = pd.read_csv(core_df_path)
factor_df = pd.read_csv(factor_path)

# time box
# start_year = 1975
start_year = 2006
end_year = 2022
df = df[(df["year"] >= start_year) & (df["year"] <= end_year)]

df = df.merge(factor_df, on=group_by_columns, how="left")
df = df.sort_values(by=["year", "area_code"])

# prepare metrics
df[metric_pct_chg] = df.groupby(granularity_columns)[metric].pct_change(fill_method="ffill").fillna(0) # fill na with 0 for visualizations

# set up target variables
df[upcoming_metric] = df.groupby(granularity_columns)[metric].shift(-1)
df[upcoming_metric_pct_chg] = df.groupby(granularity_columns)[metric_pct_chg].shift(-1)

In [111]:
df[df["area_code"] == 13101][["year", "area_code", "area", factor_column, metric_pct_chg, upcoming_metric_pct_chg]] # Tokyo-to Chiyoda-ku

,year,area_code,area,taxable_income_growth,unit_price_wmedian_pct_chg,upcoming_unit_price_wmedian_pct_chg
3099,2006,13101,Tokyo-to Chiyoda-ku,0.096990,0.000000,0.450577
3043,2007,13101,Tokyo-to Chiyoda-ku,0.106918,0.450577,-0.298286
2986,2008,13101,Tokyo-to Chiyoda-ku,0.068980,-0.298286,-0.080213
2928,2009,13101,Tokyo-to Chiyoda-ku,-0.086969,-0.080213,0.250000
2871,2010,13101,Tokyo-to Chiyoda-ku,-0.034085,0.250000,-0.400000
2816,2011,13101,Tokyo-to Chiyoda-ku,0.075307,-0.400000,-0.044261
2759,2012,13101,Tokyo-to Chiyoda-ku,-0.060839,-0.044261,0.525820
2700,2013,13101,Tokyo-to Chiyoda-ku,0.062684,0.525820,0.186464
2642,2014,13101,Tokyo-to Chiyoda-ku,0.203222,0.186464,-0.133050
2585,2015,13101,Tokyo-to Chiyoda-ku,-0.002293,-0.133050,0.187606


In [117]:
comparables = [
    13102,  # Tokyo-to Chuo-ku
    13103,  # Tokyo-to Minato-ku
    15461,  # Niigata-ken Yuzawa-Machi
    20321,  # Nagano-ken Karuisawa-machi
]

frequency_n = 500
return_n = 10
factor_n = 10

frequency_df = get_most_active_municipalities(
    df.copy(), n=frequency_n, keep=comparables
)

return_df = get_highest_growth_municipalities(
    frequency_df.copy(),
    column=metric_pct_chg,
    cumulative_column="return_growth",
    end_year=2021,
    n=return_n,
    keep=comparables,
)

return_df["factor_growth"] = get_cumulative_growth(return_df.copy(), factor_column)

factor_growth_df = get_highest_growth_municipalities(
    frequency_df.copy(),
    column=factor_column,
    cumulative_column="factor_growth",
    end_year=2021,
    n=factor_n,
    keep=comparables,
)

factor_growth_df["return_growth"] = get_cumulative_growth(
    factor_growth_df.copy(), metric_pct_chg
)

# Tokyo Area Codes are 13000 + n. Therefore, if area_code // 1000 == 13, it is in Tokyo
tokyo_df = frequency_df[frequency_df["area_code"] // 1000 == 13]
tokyo_df["return_growth"] = get_cumulative_growth(tokyo_df.copy(), metric_pct_chg)
tokyo_df["factor_growth"] = get_cumulative_growth(tokyo_df.copy(), factor_column)

In [118]:
plot_df = factor_growth_df.copy()

In [119]:
plot_time_series(
    plot_df,
    "return_growth",
    group_by_columns,
    granularity_columns,
    f"Cumulative Price Growth for top and bottom {return_n} municipalities",
    visible="legendonly",
)

In [120]:
plot_time_series(
    plot_df[plot_df["year"] <= 2020],
    "factor_growth",
    group_by_columns,
    granularity_columns,
    f"cumulative {factor_column} for top and bottom {return_n} municipalities",
    # visible="legendonly",
)

In [121]:
plot_time_series(
    plot_df,
    "count",
    group_by_columns,
    granularity_columns,
    f"Count for top and bottom {return_n} municipalities",
    # visible="legendonly",
)

In [22]:
# See if you can make a liquid long short equity strategy using REITs

# MVP
# Validating

In [ ]:
# Notebook idea
# Put in area
# See Graph of all factors
# See Graph of all prices

